# First Task

#### 1. Install BeautifulSoup4 library

We need to install beautifulsoup4 library in the first place for web scrapping in this task.

In [1]:
# conda install -c anaconda beautifulsoup4

#### 2. Import all needed libraries
All necessary library imported in this project.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

#### 3. Web Scrapping the wikipedia page containing list of neighborhoods in Toronto
The following lines of code are for web scrapping the wikipedia page that contains list of neighborhoods in the city of Toronto.

In [3]:
# URL of the wikipedia page
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# Create beautiful soup object
soup = BeautifulSoup(source, 'lxml')

# Find table that contains the neighborhoods data in the page
table = soup.find('table').tbody

# Get all columns name
columns = [item.text.replace("\n", "") for item in table.find_all("th")]

# Get all neighborhood data
neighborhoods_data = []
for i,row in enumerate(table.find_all('tr')):
    neighborhoods_data.append([])
    for j, item in enumerate(row.find_all('td')):
        neighborhoods_data[i].append(item.text.replace("\n", ""))

neighborhoods_data = neighborhoods_data[1:]
neighborhoods_data = list(map(list, zip(*neighborhoods_data)))

# Create Dataframe
tor_neigh_df = pd.DataFrame({columns[i]: neighborhoods_data[i] for i in range(3)})
tor_neigh_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 4. Rename PostalCode Column
We want to rename one column in our dataframe.

In [4]:
tor_neigh_df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 5. Drop Not Assigned value in Borough Column
All rows with not assigned borough value will be dropped.

In [5]:
tor_neigh_df = tor_neigh_df[tor_neigh_df["Borough"]!="Not assigned"]
tor_neigh_df.reset_index(drop=True, inplace=True)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 6. Cell with Not Assigned neighborhood will be filled with the same value as the borough 

In [6]:
# Function to check neighborhood value and process it if it is still not assigned
def replace_neighborhood_na(row):
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
    return row

In [7]:
tor_neigh_df = tor_neigh_df.apply(replace_neighborhood_na, axis=1)
tor_neigh_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 7. Combine rows with same PostalCode

In [8]:
neigh_df = tor_neigh_df.groupby(by="PostalCode").agg({"Borough": 'first', "Neighborhood": ", ".join})
neigh_df.reset_index(inplace=True)
neigh_df.tail()

,PostalCode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


shape of the Dataframe

In [9]:
neigh_df.shape

(103, 3)

# Second Task

#### 1. Install geocoder library
This library is needed for getting longitude and latitude of each neighborhood.

In [10]:
# conda install -c conda-forge geocoder

#### 2. Get all longitude and latitude of each neighborhood
##### a. Using Geocoder

In [11]:
# import geocoder # import geocoder

# latitudes = []
# longitudes = []

# for index, row in neigh_df.iterrows():
#     print("Getting latitude and longitude of {}, {}, {}".format(row["PostalCode"], row["Borough"], row["Neighborhood"]))
#     # initialize your variable to None
#     lat_lng_coords = None
    
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#       g = geocoder.google('{}, {}, {}'.format(row["PostalCode"], row["Borough"], row["Neighborhood"]))
#       lat_lng_coords = g.latlng

#     latitudes.append(lat_lng_coords[0])
#     longitudes.append(lat_lng_coords[1])
#     print("                                                      [DONE]")

##### b. Using available longitude and latitude data in csv format

In [12]:
ll_df = pd.read_csv("Geospatial_Coordinates.csv")
ll_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True) #rename postal code column
ll_df.set_index("PostalCode", inplace=True)
ll_df.head()

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### 3. Merge these two dataframes
We want to assign latitude and longitude of every neighborhood.

In [13]:
neigh_df.set_index("PostalCode", inplace=True)
neigh_df.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [14]:
neigh_df = neigh_df.join(ll_df)
neigh_df.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
neigh_df.reset_index(inplace=True)
neigh_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
neigh_df.shape

(103, 5)

# Third Task

#### 1. Import all necessary libraries for clustering and visualization

In [17]:
# !conda install scikit-learn

In [18]:
from sklearn.cluster import k_means
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

#### 2. Filter Neighborhood that is located in The City of Toronto 
We do this by taking only boroughs that contain the word Toronto. 

In [19]:
toronto_df = neigh_df[neigh_df["Borough"].str.contains("Toronto")].reset_index(drop=True)
print("There are {} neighborhoods in Toronto".format(toronto_df.shape[0]))
toronto_df.head()

There are 39 neighborhoods in Toronto


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### 3. Set credentials for accessing Foursquare API

In [20]:
CLIENT_ID = 'VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG' # your Foursquare ID
CLIENT_SECRET = '3QQSABWTS1WHHYFWIAB4UXPKMVDRPEKNKUHEI4SCVZSBHMM1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG
CLIENT_SECRET:3QQSABWTS1WHHYFWIAB4UXPKMVDRPEKNKUHEI4SCVZSBHMM1


#### 4. Get Nearby Venues from each Neighborhood

In [21]:
def getNearbyVenues(postalcodes, boroughs, neighborhoods, latitudes, longitudes, radius=500):
    LIMIT = 50
    venues_list=[]
    for postalcode, borough, neighborhood, lat, lng in zip(postalcodes, boroughs, neighborhoods, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except Exception as e:
            print(e)
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode,
            borough,
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Borough',
                  'Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(postalcodes=toronto_df['PostalCode'],
                                 boroughs=toronto_df["Borough"],
                                 neighborhoods=toronto_df["Neighborhood"],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

In [23]:
toronto_venues.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Remove Venues with type Neighborhood

After doing some checking, I found that there are venues with type Neighborhood. We need to distinguish neighborhood and venue to avoid confusion. So removing venues with type neighborhood will be good for our analysis.

In [24]:
toronto_venues[toronto_venues["Venue Category"] == "Neighborhood"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
76,M4M,East Toronto,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
558,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
593,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


In [25]:
toronto_venues = toronto_venues[toronto_venues["Venue Category"] != "Neighborhood"]
toronto_venues[toronto_venues["Venue Category"] == "Neighborhood"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


#### 5. One Hot Encoding

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

columns = toronto_onehot.columns

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode']
toronto_onehot['Borough'] = toronto_venues['Borough']
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ["PostalCode", "Borough", "Neighborhood"] + list(columns)
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(1204, 217)

#### 6. Find the average occurence of venues in each neighborhood

In [28]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.0,0.0,...,0.0,0.0,0.0,0.020408,0.0,0.0,0.0,0.0,0.0,0.0
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
toronto_grouped.shape

(39, 215)

#### 7. Get Top 10 Venues based on the average occurence value

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Steakhouse,Bar,Asian Restaurant,Gastropub,Sushi Restaurant,Hotel,Pizza Place,Breakfast Spot
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Steakhouse,Beer Bar,Farmers Market,Café,Liquor Store
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Pet Store,Convenience Store,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Park,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique


#### 8. Cluster Neighborhoods

In [32]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = k_means(X=toronto_grouped_clustering, n_clusters=kclusters, random_state=0)

# check cluster labels generated for each row in the dataframe
len(kmeans[1])

39

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans[1])

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Yoga Studio,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Pizza Place,Park,Coffee Shop,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Music Store,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Lawyer,Park,Swim School,Bus Line,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


#### 9. Visualize Neighborhood Clusters

In [34]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### ALL NEIGHBORHOOD CLUSTER

Neighborhood with cluster labels = 0

In [35]:
toronto_merged[toronto_merged["Cluster Labels"] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Pool,Garden,Ice Cream Shop,Yoga Studio,Cupcake Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


Neighborhood with cluster labels = 1

In [36]:
toronto_merged[toronto_merged["Cluster Labels"] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Lawyer,Park,Swim School,Bus Line,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Playground,Park,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Neighborhood with cluster labels = 2

In [37]:
toronto_merged[toronto_merged["Cluster Labels"] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Yoga Studio,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Pizza Place,Park,Coffee Shop,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Music Store,Sandwich Place
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Gym,Breakfast Spot,Clothing Store,Park,Hotel,Convenience Store,Food & Drink Shop,Sandwich Place,Dance Studio,Dumpling Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Burger Joint,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Pizza Place,Toy / Game Store,Café,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Diner
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Coffee Shop,Pub,American Restaurant,Bagel Shop,Restaurant,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Supermarket,Pizza Place
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,2,Café,Coffee Shop,Restaurant,Bakery,Park,Italian Restaurant,Pub,Pizza Place,Chinese Restaurant,Plaza
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Gym,Burger Joint,Men's Store,Japanese Restaurant,Gastropub,Coffee Shop,Gay Bar,Bubble Tea Shop,Hotel,Restaurant
